# Host a custom built BYOC inference image   
- Choose Python3 Data Science kernel in SageMaker Studio 
- Create a dummy model.tar.gz with nothing in it, and upload it to S3(Make sure SageMaker role has access to the bucket)
- Use the script in the folder to built a docker image and push it to Amazon ECR
- Run following code to create a SageMaker model with the dummy model file
- A SageMaker endpoint will be created based on the MXNet model file

In [ ]:
import sagemaker
from sagemaker.mxnet import MXNetModel
from time import gmtime, strftime
bucket_name = 'S3_BUCKET_NAME'
s3_prefix = 'MODEL_FILE_KEY_PREFIX'
model = MXNetModel(
    entry_point='entrypoint.py',
    model_data='s3://{}/{}/model.tar.gz'.format(bucket_name, s3_prefix),
    framework_version='1.6.0',
    py_version='py3',
    source_dir='inference_src',#'source_directory'
    image_uri='ACCOUNT.dkr.ecr.AWS_REGION.amazonaws.com/IMAGE_NAME',
    role=sagemaker.get_execution_role(),
    name='audio-embedding-model'+ strftime('%Y-%m-%d-%H-%M-%S', gmtime())
)

In [ ]:
ep_name = 'audio-embedding-endpoint'+ strftime('%Y-%m-%d-%H-%M-%S', gmtime())
predictor = model.deploy(initial_instance_count=1, 
                        instance_type='ml.m5.xlarge',
                        endpoint_name=ep_name)

# Test the endpoint

In [ ]:
import boto3
import json
client = boto3.client('sagemaker-runtime')
test_image = 'test_audio.mp3'
img = open(test_image, 'rb').read()
response = client.invoke_endpoint(
    EndpointName=ep_name,
    Body=img,
    ContentType='application/x-image')

result = response["Body"].read().decode("utf-8")
detections = json.loads(result)
print (detections)

In [ ]:
boto3.client('sagemaker').delete_endpoint(EndpointName=ep_name)